In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/parsed_data.csv")

In [3]:
df

,app,hostname,error_model,has_end,sdc,it,ker_time,log_file,due_cause
0,accl,carolgtxtn,ICOC,1,0,0,3.381940,log_helper_dest/radiation-benchmarks/log/2022_...,NaN
1,accl,carolgtxtn,ICOC,0,0,0,0.000000,log_helper_dest/radiation-benchmarks/log/2022_...,::ERROR FAIL in kernel execution (an illegal m...
2,accl,carolgtxtn,ICOC,1,0,0,3.349242,log_helper_dest/radiation-benchmarks/log/2022_...,NaN
3,accl,carolgtxtn,ICOC,0,0,0,0.000000,log_helper_dest/radiation-benchmarks/log/2022_...,::ERROR FAIL in kernel execution (an illegal m...
4,accl,carolgtxtn,ICOC,0,0,0,0.000000,log_helper_dest/radiation-benchmarks/log/2022_...,::ERROR FAIL in kernel execution (an illegal m...
...,...,...,...,...,...,...,...,...,...
35995,quicksort,carolgtxtn,IRA,0,0,0,0.000000,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal instruction was encountered);
35996,quicksort,carolgtxtn,IRA,0,0,0,0.000000,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal instruction was encountered);
35997,quicksort,carolgtxtn,IRA,0,0,0,0.000000,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal instruction was encountered);
35998,quicksort,carolgtxtn,IRA,0,0,0,0.000000,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal instruction was encountered);


In [8]:
due_cause_names = {
 '::ERROR FAIL in kernel execution (an illegal memory access was encountered); ': "IllegalMemAccess",
 '::ERROR FAIL in kernel execution (misaligned address); ': "MisalignedAddress",
 'TIMEOUT': "Timeout",
 '::ERROR FAIL in kernel execution (operation not supported on global/shared address space); ': "OpNotSupp.AddressSpace",
 '::ERROR FAIL in kernel execution (invalid program counter); ': "InvalidPC",
 '(an illegal memory access was encountered); ': "IllegalMemAccess",
 '(an illegal instruction was encountered); ': "IllegalInstruction",
 '(misaligned address); ':  "MisalignedAddress"
}

In [ ]:
df["due_cause"] = df["d